## OneAtlas Data API methods including authentication, contract info, and search. Methods used here are described in the One Atlas Data API Reference: https://api.oneatlas.airbus.com/api-catalog/oneatlas-data/index.html

#### Get a bearer token

In [ ]:
import requests
from json import loads, dumps
import getpass

api_key = getpass.getpass('Enter your API key: ')

url = "https://authenticate.foundation.api.oneatlas.airbus.com/auth/realms/IDP/protocol/openid-connect/token"
payload='client_id=IDP&grant_type=api_key&apikey=' + api_key
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = 'Bearer ' + (loads(response.text)["access_token"])

#### Get contract information including the workspace ID

In [ ]:
url = "https://data.api.oneatlas.airbus.com/api/v1/me"

payload={}
headers = {
  'Authorization': auth_header
}

my_info = requests.request("GET", url, headers=headers, data=payload)

contract_id = loads(my_info.text)["contract"]["id"]
workspace_id = loads(my_info.text)["contract"]["workspaceId"]

print(dumps(loads(my_info.text), indent=4, sort_keys=True))

#### Search the workspace

In [ ]:
url = "https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch"

querystring = {"itemsPerPage":100, "startPage":1, "sortBy": "-publicationDate", "workspace": workspace_id}
headers = {
    'Cache-Control': 'no-cache',
    'Authorization': auth_header, 
    'Content-Type': 'application/json'}

response = requests.request('GET', url, headers=headers, params=querystring)

print(dumps(loads(response.text), indent=4, sort_keys=True))

#### Get select information about the workspace search results that are available for download

In [ ]:
image_properties = []
image_geometries = []
image_hrefs = []
image_resource_ids =[]
image_names = []

i = 1
dl = True
rid = True
for feature in loads(response.text)['features']:
    image_properties.append(feature['properties'])
    image_geometries.append(feature['geometry'])
    try:
        if 'download' in feature['_links']:
            image_hrefs.append(feature['_links']['download'][2]['href'])
            dl = True
    except:
        dl = False
    if dl == True:
        try:
            image_resource_ids.append(feature['_links']['download'][2]['resourceId'])
            rid = True
        except:
            rid = False
    image_names.append(feature['properties']['sourceIdentifier'])
    i += 1
    dl = True
    rid = True
    
index = 0
for item in image_names:
    print(image_names[index], ' ', image_hrefs[index], ' ', image_resource_ids[index] )
    print(image_properties[index])
    print(image_geometries[index])
    print('\n')
index += 1

#### Search the One Atlas Data catalog by acquisition date range, geometry, cloud cover, incident angle, etc.

In [ ]:
url = "https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch"

payload="{\"itemsPerPage\":100,\"startPage\":1,\"productType\":\"bundle,mono\",\"acquisitionDate\":\"[2021-03-01T00:00:00.000Z,2021-04-01T23:59:59.999Z]\",\"cloudCover\":\"[0,10]\",\"incidenceAngle\":\"[0,33]\",\"sortBy\":\"-publicationDate\",\"workspace\":\"public\",\"relation\":\"intersects\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-9.176808297561358,38.728242233199325],[-9.16970342383137,38.728242233199325],[-9.16970342383137,38.73326393859847],[-9.176808297561358,38.73326393859847],[-9.176808297561358,38.728242233199325]]]}}"
headers = {
  'Authorization': auth_header,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(dumps(loads(response.text), indent=4, sort_keys=True))

#### Get additional details about the subscription, contracts, orders, deliveries, reports, etc.

In [ ]:
# Use one of these endpoints as the URL for specific details

# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/cisContracts'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/deliveries'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/orders'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/payments'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/reports'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/subscriptions'.format(contract_id)

url = 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/subscriptions'.format(contract_id)

headers = {
  'Authorization': auth_header
}

response = requests.request("GET", url, headers=headers)
print(dumps(loads(response.text), indent=4, sort_keys=True))